In [1]:
# CLASS AND FUNCTION DEFINITIONS

import random
import math

def getChromosomeScore(chromsome):
    score = chromsome.score
    return (score is not None, score) # Put scores that are not calculated at the end

###############

class Population:
    def __init__(self, size, genesCount, possibleValues, mutationRate, repopulationRate):
        self.possibleValues = possibleValues
        self.size = size
        self.mutationRate = mutationRate
        self.repopulationRate = repopulationRate
        self.chromosomes = []
        for i in range(0, size):
            self.chromosomes.append(Chromosome(genesCount, possibleValues))
    def calculateFitnessScores(self):
        population = self
        for i in range(0, population.size):
            targetChromosome = population.chromosomes[i]
            targetChromosome.calculateFitness()
    def sortPopulationByFitnessScores(self):
        population = self
        population.chromosomes.sort(key = lambda chromosome: chromosome.score, reverse=True)
    def getNormalizedScores(self):
        normalizedScores = []
        sumScore = 0
        for i in range(0, population.size):
            sumScore += population.chromosomes[i].score
        for i in range(0, population.size):
            normalizedScores.append(population.chromosomes[i].score / sumScore)
        return normalizedScores
    def getChromosomeUsingRouletteWheel(self):
        normalizedScores = self.getNormalizedScores()
        
        cursor = 0
        randomValue = random.uniform(0, 1)
        selectedIndex = None
        for i in range(0, population.size):
            cursor += normalizedScores[i]
            if(cursor >= randomValue):
                selectedIndex = i
                break
        return i
    def chooseTwoSelectedParents(self):
        
        population = self
        parentsIndexes = []
        newParentIndex = self.getChromosomeUsingRouletteWheel()
        while(newParentIndex in parentsIndexes or len(parentsIndexes) < 2):
            if(newParentIndex not in parentsIndexes):
                parentsIndexes.append(newParentIndex)
            newParentIndex = self.getChromosomeUsingRouletteWheel()
        parents = []
        for i in range(0, len(parentsIndexes)):
            parents.append(population.chromosomes[parentsIndexes[i]])
        return parents[0], parents[1]
    def makeChromosome(self, genes):
        return Chromosome(len(genes), self.possibleValues, genes)
    def addNewChildren(self, children):
        for i in range(0, len(children)):
            child = children[i];
            self.chromosomes.append(child)
        self.updateSize()
    def removeChildren(self, children):
        population = self
        for i in range(0, len(children)):
            population.chromosomes.remove(children[i])
        self.updateSize()
    def getBestPerforming(self, count):
        return sorted(population.chromosomes, key = getChromosomeScore, reverse = True)[0:count]
    def getWorstPerforming(self, count):
        return sorted(population.chromosomes, key = getChromosomeScore, reverse = False)[0:count]
    def updateSize(self):
        self.size = len(population.chromosomes)

defaultPossibleValues = [0, 1]

class Chromosome:
    def __init__(self, genesCount, possibleValues, genes=None):
        self.size = genesCount
        self.possibleValues = possibleValues
        if(genes == None):
            self.genes = []
            for i in range(0, genesCount):
                self.genes.append(possibleValues[random.randint(0, len(defaultPossibleValues) - 1)])
        else:
            self.genes = genes
        self.score = None
    def calculateFitness(self):
        chromosome = self
        chromosome.score = calculateChromosomeFitness(chromosome)
    def mutate(self, rate):
        genes = self.genes
        for i in range(0, len(genes)):
            if(random.uniform(0, 1) < rate):
                initialGeneValue = genes[i]
                newGeneVal = initialGeneValue
                while(newGeneVal == initialGeneValue):
                    newGeneVal = self.possibleValues[math.floor(random.uniform(0, 1) * len(self.possibleValues))]
                genes[i] = newGeneVal

def getCrossoverChildFromParents(parent1, parent2, crossoverPointIndexes):
    if(len(parent1) != len(parent2)):
        raise Exception("Not same lengths!")
    
    child1 = []
    child2 = []
    genesCount = len(parent1)
    cursorOnFirstParent = True
    firstChildCrossingParent = parent1
    secondChildCrossingParent = parent2
    for i in range(0, genesCount):
        if(i in crossoverPointIndexes):
            cursorOnFirstParent = not cursorOnFirstParent
            if(cursorOnFirstParent):
                firstChildCrossingParent = parent1
                secondChildCrossingParent = parent2
            else:
                firstChildCrossingParent = parent2
                secondChildCrossingParent = parent1
        child1.append(firstChildCrossingParent[i])
        child2.append(secondChildCrossingParent[i])
    return [child1, child2]

def processGeneration(population):
    
    population.calculateFitnessScores()
    
    population.sortPopulationByFitnessScores()
    
    # Making sure repopulationCount is an even number
    repopulationCount = math.floor(population.repopulationRate * population.size / 2) * 2
    
    newChildren = []
    
    for i in range(0, int(repopulationCount / 2)):
        parent1, parent2 = population.chooseTwoSelectedParents()
        parentsSize = parent1.size
        crossoverChildren = getCrossoverChildFromParents(parent1.genes, parent2.genes, [math.ceil(parentsSize / 2)])
        child1, child2 = population.makeChromosome(crossoverChildren[0]), population.makeChromosome(crossoverChildren[1])
        child1.mutate(population.mutationRate)
        child2.mutate(population.mutationRate)
        
        children = [child1, child2]
        newChildren += children
        
        population.calculateFitnessScores()
        population.sortPopulationByFitnessScores()
    
    worstPerformingChromosomes = population.getWorstPerforming(repopulationCount)
    
    population.removeChildren(worstPerformingChromosomes)
    
    population.addNewChildren(newChildren)

def stopCondition(generationNumber):
    return generationNumber >= 100

In [2]:
# LET'S RULE OUR VIRTUAL WORLD :D

# First half of the chromosome's genes have positive score and the second half have negative score
def calculateChromosomeFitness(chromosome):
    genes = chromosome.genes
    score = len(genes)
    
    for i in range(0, len(genes)):
        firstHalf = i < len(genes) / 2
        gene = genes[i]
        if(firstHalf):
            score += gene
        else:
            score -= gene
    return score

def showProperMessageForGeneration(generationNumber, bestChromosome, lastBestChromosome):
    
    bestChromosomeScore = bestChromosome.score
    bestChromosomeGenes = bestChromosome.genes

    if(lastBestChromosome == None):
        print("First generation best chromosome score: " + str(bestChromosomeScore))
        print("First generation best chromosome genes: " + str(bestChromosomeGenes))
        return
    
    lastBestChromosomeScore = lastBestChromosome.score
    lastBestChromosomeGenes = lastBestChromosome.genes
    
    if(bestChromosomeScore > lastBestChromosomeScore):
        print("Improvement in generation #" + str(generationNumber) + " best chromosome score: " + str(bestChromosomeScore))
        print("New generation best chromosome genes: " + str(bestChromosomeGenes))

MUTATION_RATE = 0.01
POPULATION_SIZE = 100
GENES_COUNT = 50
REPOPULATION_RATE = 0.9
POSSIBLE_VALUES = defaultPossibleValues

population = Population(POPULATION_SIZE, GENES_COUNT, POSSIBLE_VALUES, MUTATION_RATE, REPOPULATION_RATE)

lastBestChromosome = None

generationNumber = 0

while(not stopCondition(generationNumber)):

    processGeneration(population)
    
    bestChromosome = population.getBestPerforming(1)[0]
    
    showProperMessageForGeneration(generationNumber, bestChromosome, lastBestChromosome)
    
    lastBestChromosome = bestChromosome

    generationNumber += 1
    
print("Done")

bestChromosome = population.getBestPerforming(1)[0]

print("Best chromosome genes:", bestChromosome.genes)

First generation best chromosome score: 57
First generation best chromosome genes: [1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1]
Improvement in generation #1 best chromosome score: 60
New generation best chromosome genes: [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0]
Improvement in generation #2 best chromosome score: 61
New generation best chromosome genes: [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0]
Improvement in generation #6 best chromosome score: 62
New generation best chromosome genes: [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0]
Improvement in generation #10 best